In [0]:
%sql

WITH ranked_dups AS (
    SELECT
        uid,
        pid,
        host_id,
        offer_id,
        event_name,
        is_backfill,
        refund_withheld,
        order AS achievement_order,
        reward_amount,
        reward_usd,
        created_at,
        partition_dt,
        ROW_NUMBER() OVER (
            PARTITION BY uid, pid, event_name, reward_usd
            ORDER BY created_at
        ) AS rn
    FROM datalake_prd.silver.lp_game_achievements
    WHERE partition_dt >= DATE_SUB(current_date(), 30)
),

game_titles AS (
    SELECT uid, pid, title
    FROM silver.lp_installs
),

latest_ipqs AS (
    SELECT
        userId,
        bot_status,
        ROW_NUMBER() OVER (PARTITION BY userId ORDER BY timestamp DESC) AS ipqs_rn
    FROM datalake_prd.fraud.lp_ipqs_ip
),

earliest_ipqs AS (
    SELECT
        userId,
        bot_status,
        ROW_NUMBER() OVER (PARTITION BY userId ORDER BY timestamp ASC) AS ipqs_rn 
    FROM datalake_prd.fraud.lp_ipqs_ip
),

full_profile AS (
SELECT
    rd.uid AS user_id,
    lpu.created_at AS user_creation_date,
    rd.host_id,
    lpu.b_reason AS ban_reason,
    rd.pid AS PID,
    gt.title AS game_title,
    first_ipqs.bot_status AS first_bot_status,     
    latest_ipqs.bot_status AS latest_bot_status,   
    COUNT(*) AS achievement_count,
    SUM(CASE WHEN is_backfill = true THEN 1 ELSE 0 END) AS backfill_ct,
    SUM(CASE WHEN refund_withheld = true THEN 1 ELSE 0 END) AS withhold_ct,
    SUM(CASE WHEN refund_withheld = true THEN reward_usd ELSE 0 END) AS withheld_value,
    SUM(reward_usd) AS reward_total_USD,
    SUM(reward_usd) - SUM(CASE WHEN refund_withheld = true THEN reward_usd ELSE 0 END) AS reward_total_USD_after_withhold,
    (DATE_DIFF(MINUTE, MIN(rd.created_at), MAX(rd.created_at)) / 60.0) AS achievement_minmax_delta_hours,
    DATE_DIFF(MINUTE, MIN(rd.created_at), MAX(rd.created_at)) AS achievement_minmax_delta_minutes,
    MAX(rd.created_at) AS most_recent_achievement
FROM ranked_dups rd
LEFT JOIN silver.lp_users lpu ON rd.uid = lpu.uid
LEFT JOIN game_titles gt ON rd.uid = gt.uid AND rd.pid = gt.pid
LEFT JOIN latest_ipqs ON rd.uid = latest_ipqs.userId AND latest_ipqs.ipqs_rn = 1
LEFT JOIN earliest_ipqs first_ipqs ON rd.uid = first_ipqs.userId AND first_ipqs.ipqs_rn = 1 
WHERE 1=1
  AND rd.rn = 1
  AND lpu.b_reason IS NULL
  AND NOT (rd.pid = 'com.superplaystudios.dicedreams' AND lpu.created_at < '2025-10-01')
GROUP BY
    rd.uid,
    lpu.created_at,
    rd.host_id,
    lpu.b_reason,
    rd.pid,
    gt.title,
    first_ipqs.bot_status, 
    latest_ipqs.bot_status  
HAVING achievement_count >= 5
   AND achievement_minmax_delta_minutes <= 60
   AND reward_total_USD >= 20
   AND latest_bot_status IS true 
ORDER BY
    reward_total_USD DESC
)

select user_id as UID from full_profile
limit 20